In [ ]:
versioninfo()

In [ ]:
Threads.nthreads()

Print path of current directory:

In [ ]:
;pwd

Check that `Apophis.jl` env is correctly loaded:

In [ ]:
]st

In [ ]:
using Revise

In [ ]:
using PlanetaryEphemeris #: yr, au, c_au_per_day # these variables are already exported

In [ ]:
using Apophis

In [ ]:
using TaylorIntegration

In [ ]:
methods(TaylorIntegration.jetcoeffs!)

In [ ]:
using Plots, DelimitedFiles, Dates
using Statistics: mean, std
using Printf

In [ ]:
using SPICE

Load Sun, 8 planets, Moon, Pluto and 16 asteroid ephemeris, previously integrated with TaylorIntegration.jl

In [ ]:
using JLD
using Statistics: mean, std

Apophis orbit propagation setup:

In [ ]:
@show Threads.nthreads()

#script parameters (TODO: use ArgParse.jl instead)
const varorder = 10 # varorder is the order corresponding to the jet transport perturbation
const nv = 7 #number of TaylorN variables
const objname = "Apophis"
const maxsteps = 10000
const nyears = 5.0 #-5.0 #24.0
const dense = true #false
const apophisjlpath = dirname(pathof(Apophis))
const radarobsfile = joinpath(apophisjlpath, "../Apophis_JPL_data_2012_2013.dat")
# const radarobsfile = joinpath(apophisjlpath, "../Apophis_JPL_data_2005_2006.dat")
const dynamics = RNp1BP_pN_A_J23E_J2S_ng_eph_threads!
const jd0 = datetime2julian(DateTime(2008,9,24,0,0,0)) #starting time of integration
@show jd0 == 2454733.5
et0 = (jd0 .- J2000)*daysec
const t0 = 0.0
@show t0 == 0.0

## path to local Solar System ephemeris file
#ephfile = joinpath(apophisjlpath, "../jpleph", fnamesseph)

# dq: perturbation to nominal initial condition (Taylor1 jet transport)
dq = Taylor1.(zeros(7), varorder)
dq[end][1] = 1e-14

# # dq: perturbation to nominal initial condition (TaylorN jet transport)
# dq = set_variables("ξ", order=varorder, numvars=nv)
# for i in 1:6
#     dq[i][1][i] = 1e-8
# end
# if get_numvars() == 7
#     dq[7][1][7] = 1e-14
# end
;

Load planetary ephemeris:

In [ ]:
fnamesseph = joinpath(Apophis.jldephpath, "sseph343ast_p21y_et.jld")

# load ephemeris
ss16asteph_et = JLD.load(fnamesseph, "ss16ast_eph")
# Number of bodies
Nm1 = size(ss16asteph_et.x)[2] ÷ 6 # number of massive bodies
@show Nm1
N = Nm1 + 1 # number of bodies, including NEA
# vector of G*m values
μ_ast = Apophis.μ_ast343_DE430[1:end]
μ = vcat(Apophis.μ_DE430[1:11], μ_ast[1:Nm1-11], zero(Apophis.μ_DE430[1]))

# check: number of SS bodies (N) in ephemeris must be equal to length of GM vector (μ)
@assert N == length(μ) "Total number of bodies in ephemeris must be equal to length of GM vector μ"

# process ephemeris (switch from km,km/s units to au,au/day)
# compute Newtonian accelerations and potentials (used in post-Newtonian accelerations)
ss16asteph_auday, acc_eph, newtonianNb_Potential = Apophis.loadeph(ss16asteph_et, μ);

In [ ]:
# interaction matrix with flattened bodies
UJ_interaction = fill(false, N)
# UJ_interaction[su] = true
UJ_interaction[ea] = true
# UJ_interaction[mo] = true
params = (ss16asteph_auday, acc_eph, newtonianNb_Potential, jd0, UJ_interaction, N, μ)

# get asteroid initial conditions
q0 = Apophis.initialcond(dq)

@show tmax = t0+nyears*yr #final time of integration
;

In [ ]:
q0_old = q0
q0 = vcat(Apophis.x0_JPL_s199, 0.0) .+ dq

In [ ]:
# warmup
@time apophis_warmup = Apophis.apophisinteg(dynamics, q0, t0, tmax, Apophis.order, Apophis.abstol, params; maxsteps=5, dense=dense);

In [ ]:
#2nd warmup
@time apophis_warmup = Apophis.apophisinteg(dynamics, q0, t0, tmax, Apophis.order, Apophis.abstol, params; maxsteps=5, dense=dense);

In [ ]:
@time interp = Apophis.apophisinteg(dynamics, q0, t0, tmax, Apophis.order, Apophis.abstol, params; maxsteps=maxsteps, dense=dense);
apophis_t0 = (jd0-J2000) # days since J2000 until initial integration time
apophis_t = interp.t[:]
apophis_x = interp.x[:,:]
apophis = TaylorInterpolant(apophis_t0, apophis_t, apophis_x);

In [ ]:
julian2datetime(jd0 + apophis.t[end])

In [ ]:
apophis.t[1]

In [ ]:
apophis.t0

In [ ]:
apophis.t[1:5]

In [ ]:
ss16asteph_et.t[1:5]

In [ ]:
apophis_et(et) = Apophis.auday2kmsec(apophis(et/daysec)[1:6])
earth_et(et) = Apophis.auday2kmsec(ss16asteph_et(et)[union(3*4-2:3*4,3*(Nm1+4)-2:3*(Nm1+4))])
sun_et(et) = Apophis.auday2kmsec(ss16asteph_et(et)[union(3*1-2:3*1,3*(Nm1+1)-2:3*(Nm1+1))])

apophis_et_A2JPL(et) = apophis_et(et)(A2_JPL)
apophis_et_0(et) = apophis_et(et)()

In [ ]:
sun_et(et0)

In [ ]:
earth_et(et0)

In [ ]:
apophis_et(et0)

In [ ]:
apophis(apophis.t0)

In [ ]:
apophis(apophis.t0+4yr)

In [ ]:
apophis(apophis.t0+5yr)

Load Apophis radar astrometry, downloaded from JPL's radar astrometry website: https://ssd.jpl.nasa.gov/?radar

In [ ]:
apophis_radar_data_2005_2013 = process_radar_data_jpl("../data/Apophis_JPL_data.dat");

Construct vector of observation times in UTC and TDB:

In [ ]:
Apophis.loadjpleph()

In [ ]:
#construct vector of observation times (UTC)
tv_jpl_utc = [x.utcepoch for x in apophis_radar_data_2005_2013]
# convert to TDB
tv_jpl_tdb = str2et.(string.(tv_jpl_utc))
# date/time to Julian date
tv_jpl_tdb_julian =  J2000 .+ tv_jpl_tdb./daysec
# construct time range variable with t0 and observation times > t0, removing repeated values
tv = union(jd0, tv_jpl_tdb_julian[tv_jpl_tdb_julian .> jd0])
@show all(diff(tv) .> 0)

Take only observations from 2012-2013:

In [ ]:
apophis_data = apophis_radar_data_2005_2013[6:end]; #take only the 2012-2013 subset

Check that each receiver and transmitter antennas are the same (i.e., monostatic mode):

In [ ]:
@show all( ismonostatic.(apophis_data) )

Get the station (observatories) codes:

In [ ]:
#then, get the station codes
station_codes = [x.rcvr for x in apophis_data];

Get transmission frequencies (MHz):

In [ ]:
#extract transmitter frequencies (MHz)
transmitter_freq_MHz = [x.freq for x in apophis_data];

Get UTC receive times:

In [ ]:
# get UTC receive times
tv_jpl_utc_jul = [datetime2julian(x.utcepoch) for x in apophis_data];

Get indices of delay and Doppler observations in `apophis_data`:

In [ ]:
delay_index = findall(x->x.delay_units=="us", apophis_data)
doppler_index = findall(x->x.doppler_units=="Hz", apophis_data);

# Delay/Doppler observations reduction

In [ ]:
A2_JPL = -5.592839897872
#A2_JPL = 0.0

In [ ]:
tdelay_jpl_obs = [x.delay for x in apophis_data][delay_index]
dshift_jpl_obs = [x.doppler for x in apophis_data][doppler_index]

tdelay_jpl_obs_sigma = [x.delay_sigma for x in apophis_data][delay_index]
dshift_jpl_obs_sigma = [x.doppler_sigma for x in apophis_data][doppler_index];

In [ ]:
tc = 1.0 #90.0 #6.0

In [ ]:
# load JPL ephemeris
Apophis.loadjpleph();

In [ ]:
# compute time-delay, Doppler-shift values from JPL ephemerides
#tdelay_JPL_v1, dshift_JPL_v1 = delay_doppler(apophis_data, tc=tc, xva=Apophis.apophis_pv_197);
tdelay_JPL_v1, dshift_JPL_v1 = delay_doppler(apophis_data, tc=tc, xve=earth_et, xva=Apophis.apophis_pv_199);

In [ ]:
# Observed minus computed (O-C) residuals: JPL DE430+s199

# absolute residuals
residual_JPL_td = tdelay_jpl_obs - tdelay_JPL_v1 # (usec)
residual_JPL_ds = dshift_jpl_obs - dshift_JPL_v1 # (Hz)

# relative residuals
rel_res_JPL_td = residual_JPL_td ./ tdelay_jpl_obs # (usec)
rel_res_JPL_ds = residual_JPL_ds ./ dshift_jpl_obs # (Hz)
;

In [ ]:
@which delay_doppler(apophis_data, xve=earth_et, xvs=sun_et, xva=apophis_et, tc=tc)

In [ ]:
tdelay_JPL_v1[1]

In [ ]:
tdelay_v1[1]()

In [ ]:
tdelay_v1[1](-5.535)

In [ ]:
tdelay_jpl_obs[1]

In [ ]:
dshift_JPL_v1[2]

In [ ]:
dshift_v1[2]()

In [ ]:
dshift_v1[2](A2_JPL)

In [ ]:
dshift_jpl_obs[2]

In [ ]:
#compute time-delay and Doppler-shift "ephemeris" (i.e., predicted values according to ephemeris)
#tdelay_v1, dshift_v1 = delay_doppler(apophis_data, xve=earth_et, xvs=sun_et, xva=apophis_et, tc=tc);
tdelay_v1, dshift_v1 = delay_doppler(apophis_data, xve=earth_et, xvs=sun_et, xva=apophis_et, tc=tc);

In [ ]:
#Observed minus computed (O-C) residuals: TaylorIntegration.jl

#absolute
residual_td = tdelay_jpl_obs - tdelay_v1 # (usec)
residual_ds = dshift_jpl_obs - dshift_v1 # (Hz)

#relative
rel_res_td = residual_td ./ tdelay_jpl_obs # (usec)
rel_res_ds = residual_ds ./ dshift_jpl_obs # (Hz)
;

## Diff in residuals vs JPL sol. #197

In [ ]:
###time-delay and Doppler residuals from Brozovic et al., 2018
b2018_td_res = [0.105, 0.023, -0.491, 0.018, -0.123, -0.599, 0.121, 0.072, 0.056, 0.021, 0.059, -0.374, -0.154, 0.049, 0.400]
b2018_ds_res = [0.039, -0.023, -0.001, 0.107, -0.061, -0.052, 0.037, -0.009, -0.007, -0.027, 0.031, 0.113, -0.062, 0.010, -0.077, 0.070, -0.006, 0.009, -0.068, 0.004, 0.023, 0.016, 0.029, 0.150]
;

In [ ]:
plot(
    tv_jpl_utc_jul[delay_index].-t0,
    residual_JPL_td,
    yerror = tdelay_jpl_obs_sigma,
    m=:diamond,
    label="JPL#199 + our data proc."
)
plot!(
    tv_jpl_utc_jul[delay_index].-t0,
    residual_td(A2_JPL),
    yerror = tdelay_jpl_obs_sigma,
    m=:circle,
    label="Apophis.jl (A2 current)"
)
plot!(
    tv_jpl_utc_jul[delay_index].-t0,
    b2018_td_res,
    yerror = tdelay_jpl_obs_sigma,
    m=:square,
    label="Brozovic et al",
    leg=:bottomright
)

#ylims!(-2,2)
# earth, sun and apophis ephemeris: ours

In [ ]:
plot(
    tv_jpl_utc_jul[doppler_index].-t0,
    residual_JPL_ds,
    yerror = dshift_jpl_obs_sigma,
    m=:diamond,
    label="JPL#199 + our data proc."
)
plot!(
    tv_jpl_utc_jul[doppler_index].-t0,
    residual_ds(A2_JPL),
    yerror = dshift_jpl_obs_sigma,
    m=:circle,
    label="Apophis.jl"
)
plot!(
    tv_jpl_utc_jul[doppler_index].-t0,
    b2018_ds_res,
    yerror = dshift_jpl_obs_sigma,
    m=:square,
    label="Brozovic et al"
)
#ylims!(-2,2)
# earth, sun and apophis ephemeris: ours

# Time-delay / Doppler shift plots

In [ ]:
A2_plot = 0.0
scatter(
    tv_jpl_utc_jul[delay_index].-jd0, tdelay_v1(A2_plot),
    label="predicted (TaylorIntegration, A2 = $A2_plot x 10^-14 au/d^2)",
    marker=:cross
)
scatter!(
    tv_jpl_utc_jul[delay_index].-jd0,
    tdelay_JPL_v1,
    label="predicted (JPL DE430+s199)",
    marker=:xcross
)
scatter!(
    tv_jpl_utc_jul[delay_index].-jd0,
    tdelay_jpl_obs,
    label="observed (Goldstone, Arecibo 2012-2013)",
    legend=:topleft,
    marker=:circle,
    markersize=3,
    yerror=dshift_jpl_obs_sigma
)
title!("Time delay vs time")
ylabel!("Total time delay [useconds]")
xlims!(1540, 1645)
xticks!(1540:20:1640, string.(Date.(julian2datetime.(jd0 .+ collect(1540:20:1640)))))
xlabel!("Time")

In [ ]:
A2_plot = 0.0
scatter(
    tv_jpl_utc_jul[doppler_index].-jd0, dshift_v1(A2_plot),
    label="predicted (TaylorIntegration, A2 = $A2_plot x 10^-14 au/d^2)",
    marker=:cross
)
scatter!(
    tv_jpl_utc_jul[doppler_index].-jd0,
    dshift_JPL_v1,
    label="predicted (JPL DE430+s199)",
    marker=:xcross
)
scatter!(
    tv_jpl_utc_jul[doppler_index].-jd0,
    dshift_jpl_obs,
    label="observed (Goldstone, Arecibo 2012-2013)",
    legend=:topright,
    marker=:circle,
    markersize=3,
    yerror=dshift_jpl_obs_sigma
)
title!("Doppler shift vs time")
ylabel!("Total Doppler shift [Hz]")
xlims!(1540, 1645)
ylims!(-9e4, 8e4)
xticks!(1540:20:1640, string.(Date.(julian2datetime.(jd0 .+ collect(1540:20:1640)))))
xlabel!("Time")

In [ ]:
#A2_plot = A2_JPL # A2_lsqfit_deldop
A2_plot = 0.0
A2_label_str = @sprintf "TaylorIntegration (A2 = %.2fx10^-14 au/d^2)" A2_plot
plot(
    tv_jpl_utc_jul[delay_index].-jd0,
    #residual_JPL_td,
    b2018_td_res,
    #yerror = tdelay_jpl_obs_sigma,
    marker=:circle,
    ms=3.0,
    label = "Brozovic et al. (2018)",
    legend = :topright
)
plot!(
    tv_jpl_utc_jul[delay_index].-jd0,
    residual_td(A2_plot), 
    #yerror = tdelay_jpl_obs_sigma,
    marker=:square,
    ms=3.0,
    label = "A2 = 0.0 × 10^{-14}  au / d^{2}", #(A2 = %.2fx10^-14 au/d^2)
    legend = :topright
)
plot!(
    tv_jpl_utc_jul[delay_index].-jd0,
    residual_td(-2.0), 
    #yerror = tdelay_jpl_obs_sigma,
    marker=:square,
    ms=3.0,
    label = "A2 = -2.0 × 10^{-14}  au / d^{2}",
    legend = :topright
)
plot!(
    tv_jpl_utc_jul[delay_index].-jd0,
    residual_td(-6.0), 
    #yerror = tdelay_jpl_obs_sigma,
    marker=:square,
    ms=3.0,
    label = "A2 = -6.0 × 10^{-14}  au / d^{2}",
    legend = :topright
)
plot!(
    tv_jpl_utc_jul[delay_index].-jd0,
    residual_td(-10.0), 
    #yerror = tdelay_jpl_obs_sigma,
    marker=:square,
    ms=3.0,
    label = "A2 = -10.0 × 10^{-14}  au / d^{2}",
    legend = :topright
)
#title!("Time delay residuals (O-C)")
ylabel!("Time delay resid. (O-C) [us]")
xlims!(1540, 1645)
#ylims!(-5,4)
ylims!(-250,400)
xticks!(1540:20:1640, string.(Date.(julian2datetime.(jd0 .+ collect(1540:20:1640)))))
xlabel!("Calendar date")
#savefig("~/phdthesis/fig5-1.pdf")

In [ ]:
A2_plot = A2_JPL # A2_lsqfit_deldop
#A2_plot = 0.0 #-5.59
A2_label_str = @sprintf "TaylorIntegration (A2 = %.2fx10^-14 au/d^2)"  A2_plot
scatter(
    tv_jpl_utc_jul[delay_index].-jd0,
    residual_td(A2_plot), yerror = tdelay_jpl_obs_sigma,
    marker=:square,
    ms=3.0,
    label = A2_label_str,
    legend = :topright
)
scatter!(
    tv_jpl_utc_jul[delay_index].-jd0,
    #residual_JPL_td,
    b2018_td_res,
    yerror = tdelay_jpl_obs_sigma,
    marker=:circle,
    ms=3.0,
    label = "Brozovic et al. (2018)",
    legend = :bottomright
)
title!("Time delay residuals (O-C)")
ylabel!("Total time delay resid. (O-C) [us]")
xlims!(1540, 1645)
#ylims!(-5,4)
#ylims!(-.5,.5)
xticks!(1540:20:1640, string.(Date.(julian2datetime.(jd0 .+ collect(1540:20:1640)))))
xlabel!("Time")
#savefig("delay_residuals_A2_0.pdf")
#savefig("delay_residuals_A2_lsfit.pdf")

In [ ]:
A2_plot = 0.0
A2_label_str = @sprintf "TaylorIntegration (A2 = %.2fx10^-14 au/d^2)"  A2_plot
plot(
    tv_jpl_utc_jul[doppler_index].-jd0,
    residual_JPL_ds, yerror = dshift_jpl_obs_sigma,
    marker=:circle,
    ms=3.0,
    label = "Brozovic et al. (2018)",
    legend=:topright
)
plot!(
    tv_jpl_utc_jul[doppler_index].-jd0,
    residual_ds(0.0), yerror = dshift_jpl_obs_sigma,
    marker=:square,
    ms=2.0,
    label = "A2 = 0.0 × 10^{-14}  au / d^{2}" #A2_label_str
)
plot!(
    tv_jpl_utc_jul[doppler_index].-jd0,
    residual_ds(-2.0), yerror = dshift_jpl_obs_sigma,
    marker=:square,
    ms=2.0,
    label = "A2 = -2.0 × 10^{-14}  au / d^{2}" #A2_label_str
)
plot!(
    tv_jpl_utc_jul[doppler_index].-jd0,
    residual_ds(-6.0), yerror = dshift_jpl_obs_sigma,
    marker=:square,
    ms=2.0,
    label = "A2 = -6.0 × 10^{-14}  au / d^{2}" #A2_label_str
)
plot!(
    tv_jpl_utc_jul[doppler_index].-jd0,
    residual_ds(-10.0), yerror = dshift_jpl_obs_sigma,
    marker=:square,
    ms=2.0,
    label = "A2 = -10.0 × 10^{-14}  au / d^{2}" #A2_label_str
)
#title!("Doppler shift residuals (O-C)")
ylabel!("Doppler shift resid. (O-C) [Hz]")
xlims!(1540, 1645)
ylims!(-1.0,1.5)
xticks!(1540:20:1640, string.(Date.(julian2datetime.(jd0 .+ collect(1540:20:1640)))))
xlabel!("Calendar date")
#savefig("~/phdthesis/fig5-2.pdf")

In [ ]:
A2_plot = A2_JPL # A2_lsqfit_deldop
#A2_plot = 0.0 #-5.7
A2_label_str = @sprintf "TaylorIntegration (A2 = %.2fx10^-14 au/d^2)"  A2_plot
scatter(
    tv_jpl_utc_jul[doppler_index].-jd0,
    residual_ds(A2_plot), yerror = dshift_jpl_obs_sigma,
    marker=:square,
    ms=3.0,
    label = A2_label_str,
    leg=false
)
scatter!(
    tv_jpl_utc_jul[doppler_index].-jd0,
    residual_JPL_ds, yerror = dshift_jpl_obs_sigma,
    marker=:circle,
    ms=3.0,
    label = "JPL DE430+s199",
    legend=:bottomleft
)
title!("Doppler shift residuals (O-C)")
ylabel!("Total Doppler shift resid. (O-C) [Hz]")
xlims!(1540, 1645)
xticks!(1540:20:1640, string.(Date.(julian2datetime.(jd0 .+ collect(1540:20:1640)))))
xlabel!("Time")
#ylims!(-0.6,0.4)
#savefig("doppler_residuals_A2_0.pdf")
#savefig("doppler_residuals_A2_lsfit.pdf")

# Yarkovsky $A_2$ coefficient estimation

Here, we will compute, for a single time delay observation, the value of $A_2$ which makes the O-C residual of that observation equal to zero:

The observed value is:

In [ ]:
#observed value
tdelay_jpl_obs[1]

In [ ]:
# computed value (full solution)
tdelay_v1[delay_index[1]] # τ_c(A2) = p(A2)

In [ ]:
# residual (A2 = 0)
tdelay_jpl_obs[1] - tdelay_v1[delay_index[1]]()

In [ ]:
# substract A2=0 solution from τ_c(A2) (i.e., compute τ_c(A2)-τ_c(A2=0) )
p_τ_1 = tdelay_v1[delay_index[1]]-tdelay_v1[delay_index[1]]()

In [ ]:
# Let p_τ_1(A2) be the above polynomial expression, then we invert it so we get A2(δτ):
A2_δτ_1 = inverse(p_τ_1)

In [ ]:
#Now, we evaluate the polynomial at the corresponding residual at the A2=0 TaylorIntegration solution:
A2_δτ_1(residual_td[delay_index[1]]())

Now, we follow the same strategy for all the 2012-2013 radar astrometry for Apohpis (delay and Doppler):

In [ ]:
td_us = tdelay_v1
ds_Hz = dshift_v1;

In [ ]:
A2_δτ_v = (  inverse.( td_us-td_us() )  ) # vector of A2(δτ) polynomials at each delay observation;
A2_δν_v = (  inverse.( ds_Hz-ds_Hz() )  ) # vector of A2(δν) polynomials at each Doppler observation;

In [ ]:
A2_del_v = map((x,y)->x(y), A2_δτ_v, residual_td()); # A2(δτ) polynomials evaluated at the O-C time delay residuals;
A2_dop_v = map((x,y)->x(y), A2_δν_v, residual_ds()); # A2(δν) polynomials evaluated at the O-C Doppler shift residuals;

In [ ]:
A2_del_v

In [ ]:
A2_dop_v

In [ ]:
mean(A2_del_v), std(A2_del_v)

In [ ]:
mean(A2_dop_v), std(A2_dop_v)

In [ ]:
A2_all = vcat(A2_del_v, A2_dop_v)

In [ ]:
A2_mean_all = mean(A2_all)
A2_std_all = std(A2_all)
A2_mean_all, A2_std_all

In [ ]:
scatter(
    tv_jpl_utc_jul[delay_index][1:end-3].-jd0,
    A2_del_v[1:end-3],
    marker=:cross,
    label="A2 est. (Goldstone delay obs.)"
)
scatter!(
    tv_jpl_utc_jul[delay_index][end-2:end].-jd0,
    A2_del_v[end-2:end],
    marker=:xcross,
    label="A2 est. (Arecibo delay obs.)",
    color=3,
    leg=:topright
)
ylabel!("A2(dt) x 10^14 [au/d^2]")
xlims!(1540, 1645)
xticks!(1540:20:1645, string.(Date.(julian2datetime.(jd0 .+ collect(1540:20:1640)))))
xlabel!("Time")

In [ ]:
scatter(
    tv_jpl_utc_jul[doppler_index][1:end-7].-jd0,
    A2_dop_v[1:end-7],
    marker=:cross,
    label="A2 est. (Goldstone Doppler obs.)",
    color=2
)
scatter!(
    tv_jpl_utc_jul[doppler_index][end-6:end].-jd0,
    A2_dop_v[end-6:end],
    marker=:xcross,
    label="A2 est. (Arecibo Doppler obs.)",
    leg=:bottomleft,
    color=4
)
ylabel!("A2(df) x 10^14 [au/d^2]")
xlims!(1540, 1645)
xticks!(1540:20:1645, string.(Date.(julian2datetime.(jd0 .+ collect(1540:20:1640)))))
xlabel!("Time")

In [ ]:
scatter(
    tv_jpl_utc_jul[delay_index][1:end-3].-jd0,
    A2_del_v[1:end-3],
    marker=:cross,
    label="A2 est. (Goldstone delay obs.)"
)
scatter!(
    tv_jpl_utc_jul[doppler_index][1:end-7].-jd0,
    A2_dop_v[1:end-7],
    marker=:cross,
    label="A2 est. (Goldstone Doppler obs.)"
)
scatter!(
    tv_jpl_utc_jul[delay_index][end-2:end].-jd0,
    A2_del_v[end-2:end],
    marker=:xcross,
    label="A2 est. (Arecibo delay obs.)"
)
scatter!(
    tv_jpl_utc_jul[doppler_index][end-6:end].-jd0,
    A2_dop_v[end-6:end],
    marker=:xcross,
    label="A2 est. (Arecibo Doppler obs.)",
    leg=:bottomleft
)
ylabel!("A2 x 10^14 [au/d^2]")
xlims!(1540, 1645)
ylims!(-105,5)
xticks!(1540:20:1645, string.(Date.(julian2datetime.(jd0 .+ collect(1540:20:1640)))))
xlabel!("Time")

# Intervals

In [ ]:
#]up IntervalArithmetic

In [ ]:
#]add IntervalArithmetic
using IntervalArithmetic

In [ ]:
residual_td_interval = interval.(
    (tdelay_jpl_obs-tdelay_jpl_obs_sigma)-tdelay_v1(), 
    (tdelay_jpl_obs+tdelay_jpl_obs_sigma)-tdelay_v1()
);

In [ ]:
residual_ds_interval = interval.(
    (dshift_jpl_obs-dshift_jpl_obs_sigma)-dshift_v1(),
    (dshift_jpl_obs+dshift_jpl_obs_sigma)-dshift_v1()
);

In [ ]:
A2_del_v_interval = map((x,y)->x(y), A2_δτ_v, residual_td_interval)

In [ ]:
A2_dop_v_interval = map((x,y)->x(y), A2_δν_v, residual_ds_interval)

In [ ]:
hull(A2_dop_v_interval)

In [ ]:
A2_interval_all = vcat(A2_del_v_interval, A2_dop_v_interval)
A2_I_r = radius.(A2_interval_all)
A2_I_m = mid.(A2_interval_all);

In [ ]:
params_dA2 = (A2_I_r, A2_I_m);

In [ ]:
function distribution_A2(x, params, t)
    rv, mv = params
    sum = zero(t)
    for i in eachindex(rv)
        sum += exp( -(( (t-mv[i])/rv[i] )^2)/2 )/(rv[i]*sqrt(2pi))
    end
    return sum/length(rv)
end

function t_distribution_A2(x, params, t)
    rv, mv = params
    sum = zero(t)
    for i in eachindex(rv)
        sum += exp( -(( (t-mv[i])/rv[i] )^2)/2 )/(rv[i]*sqrt(2pi))
    end
    return t*sum/length(rv)
end

function t2_distribution_A2(x, params, t)
    rv, mv = params
    sum = zero(t)
    for i in eachindex(rv)
        sum += exp( -(( (t-mv[i])/rv[i] )^2)/2 )/(rv[i]*sqrt(2pi))
    end
    return (t^2)*sum/length(rv)
end

In [ ]:
distribution_A2(0.0, params_dA2, -10.0)

In [ ]:
t_distribution_A2(0.0, params_dA2, -10.0)

In [ ]:
t2_distribution_A2(0.0, params_dA2, -10.0)

In [ ]:
tv_a2_0, xv_a2_0 = taylorinteg(distribution_A2, 0.0, -100.0, 100.0, 25, 1e-20, params_dA2);
tv_a2_1, xv_a2_1 = taylorinteg(t_distribution_A2, 0.0, -100.0, 100.0, 25, 1e-20, params_dA2);
tv_a2_2, xv_a2_2 = taylorinteg(t2_distribution_A2, 0.0, -100.0, 100.0, 25, 1e-20, params_dA2);

In [ ]:
tv_a2_0[end], xv_a2_0[end]

In [ ]:
tv_a2_1[end], xv_a2_1[end]/xv_a2_0[end]

In [ ]:
A2_lsqfit_deldop 

In [ ]:
A2_JPL # ± 2 (10^-14 au/day^2)

In [ ]:
tv_a2_2[end], xv_a2_2[end]/xv_a2_0[end]

In [ ]:
σ_A2 = sqrt(xv_a2_2[end]-xv_a2_1[end]^2)/xv_a2_0[end]

In [ ]:
x_plot_v = collect(-6:0.001:-5)
plot( x_plot_v, map(x->distribution_A2(x, A2_I_r, A2_I_m), x_plot_v) )
#ylims!(0,0.2)

- estimate variance, mean from quadrature (-5.8, -5.4)

In [ ]:
scatter(
    tv_jpl_utc_jul[delay_index][1:end-3].-jd0,
    mid.(A2_del_v_interval[1:end-3]),
    yerror=radius.(A2_del_v_interval[1:end-3]),
    label="delay (Goldstone)",
    leg=:topright,
    marker=:cross
)
scatter!(
    tv_jpl_utc_jul[delay_index][end-2:end].-jd0,
    mid.(A2_del_v_interval[end-2:end]),
    yerror=radius.(A2_del_v_interval[end-2:end]),
    label="delay (Arecibo)",
    leg=:topright,
    marker=:xcross,
    color=3
)
#ylims!(minimum(A2_del_v)-1,0)
ylabel!("A2(dt) x 10^14 [au/d^2]")
xlims!(1540, 1645)
xticks!(1540:20:1645, string.(Date.(julian2datetime.(jd0 .+ collect(1540:20:1640)))))
xlabel!("Time")
title!("A2 estimation from polynomial inversion of delay obs.")
#savefig("A2_poly_inv_delay.pdf")

In [ ]:
scatter(
    tv_jpl_utc_jul[doppler_index][1:end-7].-jd0,
    mid.(A2_dop_v_interval[1:end-7]),
    yerror=radius.(A2_dop_v_interval[1:end-7]),
    label="Doppler (Goldstone)",
    marker=:cross,
    color=2
)
scatter!(
    tv_jpl_utc_jul[doppler_index][end-6:end].-jd0,
    mid.(A2_dop_v_interval[end-6:end]),
    yerror=radius.(A2_dop_v_interval[end-6:end]),
    label="Doppler (Arecibo)",
    marker=:xcross,
    leg=:bottomleft,
    color=4
)
ylabel!("A2(df) x 10^14 [au/d^2]")
xlims!(1540, 1645)
xticks!(1540:20:1645, string.(Date.(julian2datetime.(jd0 .+ collect(1540:20:1640)))))
xlabel!("Time")

In [ ]:
p5_3a = plot([1540, 1640], [A2_mean_all, A2_mean_all], label=A2_label_str)
scatter!(
    tv_jpl_utc_jul[doppler_index][1:end-7].-jd0,
    mid.(A2_dop_v_interval[1:end-7]),
    yerror=radius.(A2_dop_v_interval[1:end-7]),
    marker=:square,
    ms=3.0,
    label="Doppler (Goldstone)",
    color=2
)
scatter!(
    tv_jpl_utc_jul[delay_index][1:end-3].-jd0,
    mid.(A2_del_v_interval[1:end-3]),
    yerror=radius.(A2_del_v_interval[1:end-3]),
    marker=:diamond,
    ms=3.0,
    leg=:bottomleft,
    label="delay (Goldstone)",
    color=1
)
scatter!(
    tv_jpl_utc_jul[doppler_index][end-6:end].-jd0,
    mid.(A2_dop_v_interval[end-6:end]),
    yerror=radius.(A2_dop_v_interval[end-6:end]),
    marker=:square,
    ms=3.0,
    label="Doppler (Arecibo)",
    annotations=( 1625, 6, "(a)")
)
scatter!(
    tv_jpl_utc_jul[delay_index][end-2:end].-jd0,
    mid.(A2_del_v_interval[end-2:end]),
    yerror=radius.(A2_del_v_interval[end-2:end]),
    marker=:diamond,
    ms=3.0,
    leg=:bottomleft,
    label="delay (Arecibo)"
)
A2_label_str = @sprintf "A2 (mean): %.3f  x 10^-14 au/d^2" A2_mean_all
ylabel!("A2 x 10^14 [au/day^2]")
#xticks!(1540:20:1640, string.(Date.(julian2datetime.(jd0 .+ collect(1540:20:1640)))))
xticks!(1540:20:1640, fill("", length(1540:20:1640)))
#xlabel!("Calendar date")
#title!("A2 estimation from radar astrometry (Brozovic et al., 2018)")
#savefig("A2_fit_all.pdf")

In [ ]:
p5_3b = plot([1540, 1640], [A2_mean_all, A2_mean_all], label=A2_label_str)
scatter!(
    tv_jpl_utc_jul[doppler_index][1:end-7].-jd0,
    mid.(A2_dop_v_interval[1:end-7]),
    yerror=radius.(A2_dop_v_interval[1:end-7]),
    marker=:square,
    ms=3.0,
    label="Doppler (Goldstone)"
)
scatter!(
    tv_jpl_utc_jul[delay_index][1:end-3].-jd0,
    mid.(A2_del_v_interval[1:end-3]),
    yerror=radius.(A2_del_v_interval[1:end-3]),
    marker=:diamond,
    ms=3.0,
    leg=:bottomleft,
    label="delay (Goldstone)"
)
scatter!(
    tv_jpl_utc_jul[doppler_index][end-6:end].-jd0,
    mid.(A2_dop_v_interval[end-6:end]),
    yerror=radius.(A2_dop_v_interval[end-6:end]),
    marker=:square,
    ms=3.0,
    label="Doppler (Arecibo)",
    annotations=( 1625, -3.3, "(b)")
)
scatter!(
    tv_jpl_utc_jul[delay_index][end-2:end].-jd0,
    mid.(A2_del_v_interval[end-2:end]),
    yerror=radius.(A2_del_v_interval[end-2:end]),
    marker=:diamond,
    ms=3.0,
    leg=:bottomright,
    label="delay (Arecibo)"
)
A2_label_str = @sprintf "A2 (mean): %.3f  x 10^-14 au/d^2" A2_mean_all
ylabel!("A2 x 10^14 [au/day^2]")
xticks!(1540:20:1640, string.(Date.(julian2datetime.(jd0 .+ collect(1540:20:1640)))))
xlabel!("Calendar date")
ylims!(-8,-3)
#title!("A2 estimation from radar astrometry (Brozovic et al., 2018)")
#savefig("A2_fit_all.pdf")

In [ ]:
p5_3 = plot(p5_3a, p5_3b, size=(640,860), layout=@layout([a; b]))
#savefig("~/phdthesis/fig5-3.pdf")

## Least squares

In [ ]:
res_del = tdelay_jpl_obs .- tdelay_v1 # delay residuals a.a.f. of A2
res_dop = dshift_jpl_obs .- dshift_v1 # Doppler residuals a.a.f. of A2
W_del = 1 ./(tdelay_jpl_obs_sigma.^2) # delay weights
W_dop = 1 ./(dshift_jpl_obs_sigma.^2) # Doppler weights
res_deldop = vcat(res_del, res_dop) # delay + Doppler residuals as a function of A2
W_deldop = vcat(W_del, W_dop) # delay + Doppler weights
res_xWx_del = res_del .* W_del .* res_del
res_xWx_dop = res_dop .* W_dop .* res_dop
res_xWx_deldop = res_deldop .* W_deldop .* res_deldop;

In [ ]:
Apophis.taylor_roots.(res_del, 0.0)

In [ ]:
Apophis.taylor_roots.(res_dop, 0.0)

scatter(
    tv_jpl_utc_jul[delay_index].-t0,
    taylor_roots.(res_del, 0.0),
    yerror=radius.(A2_del_v_interval)
)

Below, we compute the $A_2$ value which minimizes the polynomial expression for $Q(A_2)$, the mean square residual. We consider 4 cases: 1) only the delay observations, 2) only the Doppler observations, 3) all the delays and Dopplers, and 4) all delays and Dopplers excluding Arecibo Dopplers. Further, following classical nonlinear squares procedures, we estimate the 1-$\sigma$ formal uncertainty of $A_2$ for each case, from the covariance "matrix" as $\sigma_{A_2} = \Gamma_{1,1}$ where the covariance matrix is $\Gamma = C^{-1}$, $C$ is the normal matrix $C = (\partial\xi/\partial A_2)^\dagger \cdot (\partial\xi/\partial A_2)$ and $\xi$ are the residuals. In this case, since were are fitting only 1 parameter, namely $A_2$, both $\Gamma$ and $C$ correspond to 1x1 matrices (i.e., both $\Gamma$ and $C$ are scalars).

In [ ]:
nobs_del = length(res_xWx_del)
Q_A2_del = sum(res_xWx_del)/nobs_del
#res_sq_del_ord2 = map(x->Taylor1(x.coeffs[1:3]), res_sq_del)
#Q_A2_del = sum(res_sq_del_ord2)/length(res_sq_del_ord2)

In [ ]:
A2_lsqfit_del = Apophis.taylor_minimum(Q_A2_del, 0.0, niters=5)

In [ ]:
sqrt(Q_A2_del(A2_lsqfit_del)) #rms

In [ ]:
C_del = differentiate(Q_A2_del, 2)/(2/nobs_del) # C = ∂²Q /(2/nobs)
Γ_del = inv(C_del) # Γ = C⁻¹

In [ ]:
Γ_del(A2_lsqfit_del)

In [ ]:
sqrt( Γ_del(A2_lsqfit_del) )

In [ ]:
nobs_dop = length(res_xWx_dop)
Q_A2_dop = sum(res_xWx_dop)/nobs_dop

In [ ]:
A2_lsqfit_dop = Apophis.taylor_minimum(Q_A2_dop, 0.0, niters=5)

In [ ]:
sqrt(Q_A2_dop(A2_lsqfit_dop)) #rms

In [ ]:
C_dop = differentiate(Q_A2_dop, 2)/(2/nobs_dop) # C = ∂²Q /(2/nobs)
Γ_dop = inv(C_dop) # Γ = C⁻¹

In [ ]:
Γ_dop(A2_lsqfit_dop)

In [ ]:
sqrt( Γ_dop(A2_lsqfit_dop) )

In [ ]:
nobs_deldop = length(res_xWx_deldop)
Q_A2_deldop = sum(res_xWx_deldop)/nobs_deldop

In [ ]:
A2_lsqfit_deldop = Apophis.taylor_minimum(Q_A2_deldop, 0.0, niters=5)

In [ ]:
sqrt(Q_A2_deldop(A2_lsqfit_deldop)) #rms

In [ ]:
nobs_deldop

In [ ]:
C_deldop = differentiate(Q_A2_deldop, 2)/(2/(14000 + 0nobs_deldop)) # C = ∂²Q /(2/nobs)
Γ_deldop = inv(C_deldop) # Γ = C⁻¹

In [ ]:
Γ_deldop(A2_lsqfit_deldop)

In [ ]:
sqrt( Γ_deldop(A2_lsqfit_deldop) )

In [ ]:
_A2_min_plot = -5.8-0.5
_A2_max_plot = -5.4+0.5
A2_vals = range(_A2_min_plot, length=2000, stop=_A2_max_plot)
plot(A2_vals, Q_A2_deldop.(A2_vals), leg=false)
xlabel!("A2 x 10^14 [au/day^2]")
ylabel!("Q(A2)")
#title!("Mean weighted square residual vs A2")
#ylims!(-1, 11)
xlims!(_A2_min_plot, _A2_max_plot)
#savefig("~/phdthesis/fig5-4.pdf")

In [ ]:
mean(res_deldop), std(res_deldop)

Compute the value of A2 such that the mean of the residuals is zero:

In [ ]:
A2_zero_mean_res = Apophis.taylor_roots(mean(res_deldop), A2_lsqfit_deldop)

Check that the vector of residuals evaluated at the LS solution has mean zero:

In [ ]:
mean(res_deldop()), std(res_deldop()) #A2 = 0

In [ ]:
# A2 = A2 LS solution
mean(res_deldop(A2_lsqfit_deldop)), std(res_deldop(A2_lsqfit_deldop))

In [ ]:
# A2 such that res. mean is closest to zero
mean(res_deldop(A2_zero_mean_res)), std(res_deldop(A2_zero_mean_res))

If we expand the "cost" function Q wrt the solution value, then the first-order term should be zero:

In [ ]:
ΔQ_del = Q_A2_del(A2_lsqfit_del+Taylor1(Q_A2_del.order))-Q_A2_del(A2_lsqfit_del)

In [ ]:
ΔQ_deldop = Q_A2_deldop(A2_lsqfit_deldop+Taylor1(Q_A2_deldop.order))-Q_A2_deldop(A2_lsqfit_deldop)

## Ephemerides: DE430+s199 vs TaylorIntegration

In [ ]:
(tv[end]-tv[1])/365.25

In [ ]:
#SPICE.furnsh("../jpleph/naif0012.tls")
#SPICE.furnsh("../../jpleph/de430.bsp", "../jpleph/a99942_s197.bsp", "../jpleph/a99942_s199.bsp", "../jpleph/ast343de430.bsp")

In [ ]:
tv_et = Array{Float64}(undef, length(tv))
apophis_jpl197 = Array{Float64}(undef, length(tv), 6)
apophis_jpl199 = Array{Float64}(undef, length(tv), 6)
earth_jpl = Array{Float64}(undef, length(tv), 6)
moon_jpl = Array{Float64}(undef, length(tv), 6)
for i in eachindex(tv)
    tv_et[i] = julian2etsecs(tv[i])
    apophis_jpl199[i,:] .= Apophis.kmsec2auday( Apophis.apophis_pv_199(tv_et[i]) )
    apophis_jpl197[i,:] .= Apophis.kmsec2auday( Apophis.apophis_pv_197(tv_et[i]) )
    earth_jpl[i,:] .= Apophis.kmsec2auday( Apophis.earth_pv(tv_et[i]) )
    moon_jpl[i,:] .= Apophis.kmsec2auday( Apophis.moon_pv(tv_et[i]) )
end

In [ ]:
apophis.t0, apophis.t[1:2], (tv[1:2].-jd0)/86400

In [ ]:
#A2_lsqfit_deldop = A2_lsqfit_del
apophis_ti = [apophis(tv[i]-J2000)[j](A2_JPL) for i in eachindex(tv), j in 1:6];
#apophis_ti = [apophis(tv[i])[j]() for i in eachindex(tv), j in 1:6];
earth_ti = [ss16asteph_et(tv_et[i])[j] for i in eachindex(tv), j in union(3*4-2:3*4,3*(27+4)-2:3*(27+4))];
moon_ti = [ss16asteph_et(tv_et[i])[j] for i in eachindex(tv), j in union(3*5-2:3*5,3*(27+5)-2:3*(27+5))];

In [ ]:
1e3*au*(apophis_ti - apophis_jpl199) #m, m/day

In [ ]:
1e3au*(earth_ti - earth_jpl) #m, m/day

In [ ]:
1e3au*(moon_ti - moon_jpl) ##m, m/day

In [ ]:
x_ae_ti = apophis_ti - earth_ti;

In [ ]:
x_ae_jpl = apophis_jpl199 - earth_jpl;

In [ ]:
r(x,y,z) = sqrt(x^2 + y^2 + z^2)
vr(x,y,z,u,v,w) = (x*u + y*v + z*w)/r(x,y,z)

r_ae_ti = r.(x_ae_ti[:,1], x_ae_ti[:,2], x_ae_ti[:,3])
vr_ae_ti = vr.( x_ae_ti[:,1], x_ae_ti[:,2], x_ae_ti[:,3], x_ae_ti[:,4], x_ae_ti[:,5], x_ae_ti[:,6] )

r_ae_jpl = r.(x_ae_jpl[:,1], x_ae_jpl[:,2], x_ae_jpl[:,3])
vr_ae_jpl = vr.( x_ae_jpl[:,1], x_ae_jpl[:,2], x_ae_jpl[:,3], x_ae_jpl[:,4], x_ae_jpl[:,5], x_ae_jpl[:,6] );

In [ ]:
#tve = range(tv[1], tv[end], length=2000)
#tvemin, tvemax = minmax(ss16asteph_et.t0+ss16asteph_et.t[1], ss16asteph_et.t0+ss16asteph_et.t[end])
tvemin, tvemax = minmax((apophis.t0+apophis.t[1])*daysec, (apophis.t0+apophis.t[end])*daysec)
tve = range(tvemin+1, tvemax-1, length=2000)#2000)
#tve = range(ss16asteph.t[1], ss16asteph.t[1]+yr, length=50000)

In [ ]:
julian2datetime.(J2000 .+ tve./86400)

In [ ]:
sun_ti_e = [ss16asteph_et(tve[i])[j] for i in eachindex(tve), j in union(3*1-2:3*1,3*(27+1)-2:3*(27+1))]
sun_jpl_e = Array{Float64}(undef, length(tve), 6)
mercury_ti_e = [ss16asteph_et(tve[i])[j] for i in eachindex(tve), j in union(3*2-2:3*2,3*(27+2)-2:3*(27+2))]
mercury_jpl_e = Array{Float64}(undef, length(tve), 6)
venus_ti_e = [ss16asteph_et(tve[i])[j] for i in eachindex(tve), j in union(3*3-2:3*3,3*(27+3)-2:3*(27+3))]
venus_jpl_e = Array{Float64}(undef, length(tve), 6)
earth_ti_e = [ss16asteph_et(tve[i])[j] for i in eachindex(tve), j in union(3*4-2:3*4,3*(27+4)-2:3*(27+4))]
earth_jpl_e = Array{Float64}(undef, length(tve), 6)
moon_ti_e = [ss16asteph_et(tve[i])[j] for i in eachindex(tve), j in union(3*5-2:3*5,3*(27+5)-2:3*(27+5))]
moon_jpl_e = Array{Float64}(undef, length(tve), 6)
#ceres_ti_e = [ss16asteph_et(tve[i])[j] for i in eachindex(tve), j in union(3*12-2:3*12,3*(27+12)-2:3*(27+12))]
#ceres_jpl_e = Array{Float64}(undef, length(tve), 6)
mars_ti_e = [ss16asteph_et(tve[i])[j] for i in eachindex(tve), j in union(3*6-2:3*6,3*(27+6)-2:3*(27+6))]
mars_jpl_e = Array{Float64}(undef, length(tve), 6)
jupiter_ti_e = [ss16asteph_et(tve[i])[j] for i in eachindex(tve), j in union(3*7-2:3*7,3*(27+7)-2:3*(27+7))]
jupiter_jpl_e = Array{Float64}(undef, length(tve), 6)
saturn_ti_e = [ss16asteph_et(tve[i])[j] for i in eachindex(tve), j in union(3*8-2:3*8,3*(27+8)-2:3*(27+8))]
saturn_jpl_e = Array{Float64}(undef, length(tve), 6)
pluto_ti_e = [ss16asteph_et(tve[i])[j] for i in eachindex(tve), j in union(3*11-2:3*11,3*(27+11)-2:3*(27+11))]
pluto_jpl_e = Array{Float64}(undef, length(tve), 6)
for i in eachindex(tve)
    tve_et_i = tve[i]
    sun_jpl_e[i,:] .= Apophis.kmsec2auday( Apophis.sun_pv(tve_et_i) )
    mercury_jpl_e[i,:] .= Apophis.kmsec2auday( Apophis.getpv(1, 0, tve_et_i) )
    venus_jpl_e[i,:] .= Apophis.kmsec2auday( Apophis.getpv(2, 0, tve_et_i) )
    earth_jpl_e[i,:] .= Apophis.kmsec2auday( Apophis.earth_pv(tve_et_i) )
    moon_jpl_e[i,:] .= Apophis.kmsec2auday( Apophis.moon_pv(tve_et_i) )
    #ceres_jpl_e[i,:] .= Apophis.kmsec2auday( Apophis.getpv(2000001, 0, tve_et_i) )
    mars_jpl_e[i,:] .= Apophis.kmsec2auday( Apophis.getpv(4, 0, tve_et_i) )
    jupiter_jpl_e[i,:] .= Apophis.kmsec2auday( Apophis.getpv(5, 0, tve_et_i) )
    saturn_jpl_e[i,:] .= Apophis.kmsec2auday( Apophis.getpv(6, 0, tve_et_i) )
    pluto_jpl_e[i,:] .= Apophis.kmsec2auday( Apophis.getpv(9, 0, tve_et_i) )
end

In [ ]:
apophis_ti_e = [apophis(tve[i]./86400)[j]( A2_JPL ) for i in eachindex(tve), j in 1:6];
apophis_ti_e_ = [apophis(tve[i]./86400)[j]( A2_lsqfit_del ) for i in eachindex(tve), j in 1:6];
#apophis_ti_e = [apophis_p24y(tve[i]./86400)[j]( A2_JPL ) for i in eachindex(tve), j in 1:6];
#apophis_ti_e_ = [apophis_p24y(tve[i]./86400)[j]( A2_lsqfit_del ) for i in eachindex(tve), j in 1:6];

apophis_jpl197_e = Array{Float64}(undef, length(tve), 6)
apophis_jpl199_e = Array{Float64}(undef, length(tve), 6)
for i in eachindex(tve)
    tve_et_i = tve[i]
    apophis_jpl199_e[i,:] .= Apophis.kmsec2auday( Apophis.apophis_pv_199(tve_et_i) )
    apophis_jpl197_e[i,:] .= Apophis.kmsec2auday( Apophis.apophis_pv_197(tve_et_i) )
end

In [ ]:
plot( tve, 1000au*(apophis_ti_e .- apophis_jpl199_e)[:,1], label="x")#, marker=:cross)
plot!(tve, 1000au*(apophis_ti_e .- apophis_jpl199_e)[:,2], label="y")#, marker=:cross)
plot!(tve, 1000au*(apophis_ti_e .- apophis_jpl199_e)[:,3], label="z", leg=:bottomleft)#, marker=:cross)
title!("Apophis: diff. in barycentric position vs JPL#197 (A_2=0)")
ylabel!("Apophis.jl - JPL#197 [m]")
xlabel!("Calendar date (TDB)")
xticks!(tvemin:daysec*5yr:tvemax, string.(Date.(julian2datetime.((J2000+tvemin/86400):5yr:(J2000+tvemax/86400)))))
#ylims!(-500000,500000)

In [ ]:
plot( tve, 1000au*(apophis_ti_e_ .- apophis_jpl199_e)[:,1], label="x")#, marker=:cross)
plot!(tve, 1000au*(apophis_ti_e_ .- apophis_jpl199_e)[:,2], label="y")#, marker=:cross)
plot!(tve, 1000au*(apophis_ti_e_ .- apophis_jpl199_e)[:,3], label="z", leg=:bottomleft)#, marker=:cross)
title!("Apophis: diff. in barycentric position vs JPL#199")
ylabel!("Apophis.jl - JPL#199 [m]")
xlabel!("Julian date (TDB)")
xticks!(tvemin:5daysec*yr:tvemax, string.(Date.(julian2datetime.((J2000+tvemin/86400):5yr:(J2000+tvemax/86400)))))
ylims!(-500,500)

In [ ]:
plot( tve, 1000au*(apophis_ti_e_ .- apophis_jpl199_e)[:,1], label="x")#, marker=:cross)
plot!(tve, 1000au*(apophis_ti_e_ .- apophis_jpl199_e)[:,2], label="y")#, marker=:cross)
plot!(tve, 1000au*(apophis_ti_e_ .- apophis_jpl199_e)[:,3], label="z", leg=:bottomleft)#, marker=:cross)
title!("Apophis: diff. in barycentric position vs JPL#199")
ylabel!("Apophis.jl - JPL#199 [m]")
xlabel!("Julian date (TDB)")
xticks!(tvemin:5daysec*yr:tvemax, string.(Date.(julian2datetime.((J2000+tvemin/86400):5yr:(J2000+tvemax/86400)))))
#ylims!(-500,500)

In [ ]:
plot( tve, (1e6au/daysec)*(apophis_ti_e .- apophis_jpl199_e)[:,4], label="x")#, marker=:cross)
plot!(tve, (1e6au/daysec)*(apophis_ti_e .- apophis_jpl199_e)[:,5], label="y")#, marker=:cross)
plot!(tve, (1e6au/daysec)*(apophis_ti_e .- apophis_jpl199_e)[:,6], label="z", leg=:bottomleft)#, marker=:cross)
title!("Apophis: diff. in barycentric position vs JPL#199")
ylabel!("Apophis.jl - JPL#199 [mm/s]")
xlabel!("Julian date (TDB)")
xticks!(tvemin:5daysec*yr:tvemax, string.(Date.(julian2datetime.((J2000+tvemin/86400):5yr:(J2000+tvemax/86400)))))
#ylims!(-500,500)

In [ ]:
PlanetaryEphemeris.RE

- check ~2017 close approach with Venus?

In [ ]:
plot( tve, 1000au*(apophis_ti_e .- apophis_jpl199_e)[:,1], label="x")#, marker=:cross)
plot!(tve, 1000au*(apophis_ti_e .- apophis_jpl199_e)[:,2], label="y")#, marker=:cross)
plot!(tve, 1000au*(apophis_ti_e .- apophis_jpl199_e)[:,3], label="z", leg=:bottomleft)#, marker=:cross)
title!("Apophis: diff. in barycentric position vs JPL#199")
ylabel!("Apophis.jl - JPL#199 [m]")
xlabel!("Julian date (TDB)")
xticks!(tvemin:daysec*yr:tvemax, string.(Date.(julian2datetime.((J2000+tvemin/86400):yr:(J2000+tvemax/86400)))))

In [ ]:
indastn = 13
ast2_ti_e = [ss16asteph(tve[i])[j] for i in eachindex(tve), j in union(3*indastn-2:3*indastn,3*(27+indastn)-2:3*(27+indastn))]
indastn = 14
ast3_ti_e = [ss16asteph(tve[i])[j] for i in eachindex(tve), j in union(3*indastn-2:3*indastn,3*(27+indastn)-2:3*(27+indastn))]
indastn = 15
ast4_ti_e = [ss16asteph(tve[i])[j] for i in eachindex(tve), j in union(3*indastn-2:3*indastn,3*(27+indastn)-2:3*(27+indastn))]
indastn = 26
ast15_ti_e = [ss16asteph(tve[i])[j] for i in eachindex(tve), j in union(3*indastn-2:3*indastn,3*(27+indastn)-2:3*(27+indastn))]
indastn = 27
ast16_ti_e = [ss16asteph(tve[i])[j] for i in eachindex(tve), j in union(3*indastn-2:3*indastn,3*(27+indastn)-2:3*(27+indastn))]
;

In [ ]:
xap = apophis_ti_e[:,1]
yap = apophis_ti_e[:,2]
zap = apophis_ti_e[:,3]

xsu = sun_ti_e[:,1]
ysu = sun_ti_e[:,2]
zsu = sun_ti_e[:,3]

xea = earth_ti_e[:,1]
yea = earth_ti_e[:,2]
zea = earth_ti_e[:,3]

xpl = pluto_ti_e[:,1]
ypl = pluto_ti_e[:,2]
zpl = pluto_ti_e[:,3]

xce = ceres_ti_e[:,1]
yce = ceres_ti_e[:,2]
zce = ceres_ti_e[:,3]

xast2 = ast2_ti_e[:,1]
yast2 = ast2_ti_e[:,2]
zast2 = ast2_ti_e[:,3]

xast3 = ast3_ti_e[:,1]
yast3 = ast3_ti_e[:,2]
zast3 = ast3_ti_e[:,3]

xast4 = ast4_ti_e[:,1]
yast4 = ast4_ti_e[:,2]
zast4 = ast4_ti_e[:,3]

xast15 = ast15_ti_e[:,1]
yast15 = ast15_ti_e[:,2]
zast15 = ast15_ti_e[:,3]

xast16 = ast16_ti_e[:,1]
yast16 = ast16_ti_e[:,2]
zast16 = ast16_ti_e[:,3]

dasu = sqrt.((xap.-xsu).^2 .+ (yap.-ysu).^2 .+ (zap.-zsu).^2)
dapl = sqrt.((xap.-xpl).^2 .+ (yap.-ypl).^2 .+ (zap.-zpl).^2)
dace = sqrt.((xap.-xce).^2 .+ (yap.-yce).^2 .+ (zap.-zce).^2)
daast2 = sqrt.((xap.-xast2).^2 .+ (yap.-yast2).^2 .+ (zap.-zast2).^2)
daast3 = sqrt.((xap.-xast3).^2 .+ (yap.-yast3).^2 .+ (zap.-zast3).^2)
daast4 = sqrt.((xap.-xast4).^2 .+ (yap.-yast4).^2 .+ (zap.-zast4).^2)
daast15 = sqrt.((xap.-xast15).^2 .+ (yap.-yast15).^2 .+ (zap.-zast15).^2)
daast16 = sqrt.((xap.-xast16).^2 .+ (yap.-yast16).^2 .+ (zap.-zast16).^2)

a_Yark = abs.((A2_JPL*1e-14) ./ (dasu.^2))
a_G_pl = abs.(μ[11]./(dapl.^2))
a_G_ce = abs.(μ[12]./(dace.^2))
a_G_ast2 = abs.(μ[13]./(daast2.^2))
a_G_ast3 = abs.(μ[14]./(daast3.^2))
a_G_ast4 = abs.(μ[15]./(daast4.^2))
a_G_ast15 = abs.(μ[11+15]./(daast15.^2))
a_G_ast16 = abs.(μ[11+16]./(daast16.^2))
;

In [ ]:
plot( tve, 1e14(a_Yark), label="Yarkovsky acc.")
plot!(tve, 1e14(a_G_ce), label="Ceres") #, leg=:bottomleft)
plot!(tve, 1e14(a_G_ast2), label="Vesta")
plot!(tve, 1e14(a_G_ast3), label="Pallas")
plot!(tve, 1e14(a_G_ast4), label="Hygiea")
plot!(tve, 1e14(a_G_pl), label="Pluto")
#plot!(tve, 1e14(a_G_ast15), label="Patientia")
#plot!(tve, 1e14(a_G_ast16), label="Sylvia")
#title!("Accelerations: Yarkovsky, Pluto, asteroids")
ylabel!("acceleration mag. [ x10^{14} au day^{-2}]")
xlabel!("Calendar date")
#yticks!(0.0:0.25:10.0)
xticks!(tvemin:4daysec*yr:tvemax, string.(Date.(julian2datetime.((J2000+tvemin/86400):4yr:(J2000+tvemax/86400)))))
#savefig("~/phdthesis/fig5-5.pdf")

In [ ]:
#plotly()

In [ ]:
gc_range_ti_e = 1e3au*sqrt.((apophis_ti_e[:,1] .- earth_ti_e[:,1]).^2 .+ (apophis_ti_e[:,2] .- earth_ti_e[:,2]).^2 .+ (apophis_ti_e[:,3] .- earth_ti_e[:,3]).^2)
gc_range_jpl_e = 1e3au*sqrt.((apophis_jpl199_e[:,1] .- earth_jpl_e[:,1]).^2 .+ (apophis_jpl199_e[:,2] .- earth_jpl_e[:,2]).^2 .+ (apophis_jpl199_e[:,3] .- earth_jpl_e[:,3]).^2);
diff_gc_range_e = gc_range_jpl_e .- gc_range_ti_e;

bc_range_ti_e = 1e3au*sqrt.(apophis_ti_e[:,1].^2 .+ apophis_ti_e[:,2].^2 .+ apophis_ti_e[:,3].^2)
bc_range_jpl_e = 1e3au*sqrt.(apophis_jpl199_e[:,1].^2 .+ apophis_jpl199_e[:,2].^2 .+ apophis_jpl199_e[:,3].^2)
diff_bc_range_e = bc_range_jpl_e .- bc_range_ti_e;

In [ ]:
plot( tve, gc_range_ti_e)
plot!( tve, gc_range_jpl_e)
title!("Apophis: geocentric range")
ylabel!("r_A-r_E [m]")
xlabel!("Julian date (TDB)")
xticks!(tvemin:4daysec*yr:tvemax, string.(Date.(julian2datetime.((J2000+tvemin/86400):4yr:(J2000+tvemax/86400)))))

In [ ]:
plot( tve, bc_range_ti_e)
plot!( tve, bc_range_jpl_e)
title!("Apophis: barycentric range")
ylabel!("r_A(JPL#199)-r_A(Apophis.jl) [m]")
xlabel!("Julian date (TDB)")
xticks!(tvemin:4daysec*yr:tvemax, string.(Date.(julian2datetime.((J2000+tvemin/86400):4yr:(J2000+tvemax/86400)))))

In [ ]:
#plotly()

In [ ]:
plot( tve, diff_gc_range_e, label="r")#, marker=:cross)
title!("Apophis: diff. in geocentric range vs JPL#199")
ylabel!("Apophis.jl - JPL#199 [m]")
xlabel!("Julian date (TDB)")
xticks!(tvemin:4daysec*yr:tvemax, string.(Date.(julian2datetime.((J2000+tvemin/86400):4yr:(J2000+tvemax/86400)))))
ylims!(-500,500)

In [ ]:
plot( tve, (1e6au/86400)*(apophis_ti_e .- apophis_jpl199_e)[:,4], label="vx") #, marker=:cross)
plot!(tve, (1e6au/86400)*(apophis_ti_e .- apophis_jpl199_e)[:,5], label="vy") #, marker=:cross)
plot!(tve, (1e6au/86400)*(apophis_ti_e .- apophis_jpl199_e)[:,6], label="vz", leg=:bottomleft) #, marker=:cross)
title!("Apophis: diff. in barycentric velocity vs JPL#199")
ylabel!("Apophis.jl - DE430 [mm/s]")
xlabel!("Julian date (TDB)")
#ylims!(-0.0002, 0.0002)
xlims!(tvemin, tvemax)
xticks!(tvemin:4daysec*yr:tvemax, string.(Date.(julian2datetime.((J2000+tvemin/86400):4yr:(J2000+tvemax/86400)))))
ylims!(-0.1,0.1)

## Propagation up to 2021/2029

In [ ]:
@time sol_p24y = Apophis.apophisinteg(dynamics, Apophis.rvelea, q0, t0, t0+24yr, Apophis.order, Apophis.abstol, params; maxsteps=maxsteps);

In [ ]:
sol = (
    tv1 = sol_p24y[1][:],
    xv1 = sol_p24y[2][:,:],
    tvS1 = sol_p24y[3][:],
    xvS1 = sol_p24y[4][:,:],
    gvS1 = sol_p24y[5][:]
);

In [ ]:
using LinearAlgebra: norm, dot, cross

In [ ]:
julian2datetime.( jd0 .+ sol.tvS1(-5.602) )[end-30:end]

In [ ]:
# Apophis geocentric range at time t
function rea(x, params, t)
    ss16asteph_t = params[1](Taylor1(t.coeffs, 30))
    xe = ss16asteph_t[union(3ea-2:3ea,3(Apophis.N-1+ea)-2:3(Apophis.N-1+ea))]
    return sqrt( (x[1]-xe[1])^2 + (x[2]-xe[2])^2 + (x[3]-xe[3])^2 )
end

In [ ]:
map(  x->rea(sol.xvS1[x,:], params, sol.tvS1[x]), (length(sol.tvS1)-30):length(sol.tvS1)  )(A2_lsqfit_deldop)

In [ ]:
#geocentric range at closest approach (Earth radii): October 2020
au*rea(sol.xvS1[27,:], params, sol.tvS1[27])/PlanetaryEphemeris.RE

In [ ]:
#geocentric range at closest approach (Earth radii): March 2021
au*rea(sol.xvS1[29,:], params, sol.tvS1[29])/PlanetaryEphemeris.RE

In [ ]:
#geocentric range at closest approach (Earth radii): April 2029
au*rea(sol.xvS1[47,:], params, sol.tvS1[47])(A2_lsqfit_deldop)/PlanetaryEphemeris.RE

In [ ]:
#diff. in geocentric range wrt #199 at closest approach (km): April 2029
au*(rea(sol.xvS1[47,:], params, sol.tvS1[47])(A2_JPL)-0.000252172816142565)

In [ ]:
#solution #199 nominal geocentric range at April 2029 closest approach
d_CA_2029_JPL199 = 0.000252172816142565

In [ ]:
apophis_2029 = sol.xvS1[47,1:6];

In [ ]:
t_2029 = sol.tvS1[47];

In [ ]:
julian2datetime(jd0+t_2029(A2_JPL))

In [ ]:
ss16asteph_2029 = params[1](t_2029)
xe_2029 = ss16asteph_2029[union(3ea-2:3ea,3(Apophis.N-1+ea)-2:3(Apophis.N-1+ea))];

In [ ]:
xae_2029 = apophis_2029-xe_2029; # Apophis geocentric position/velocity (au, au/day)

In [ ]:
# Apophis geocentric range at 2029 closest approach (au)
rae_2029 = sqrt(xae_2029[1]^2 + xae_2029[2]^2 + xae_2029[3]^2) #(A2_JPL)

In [ ]:
σ_A2_JPL = 2.201

In [ ]:
# 3*σ_A2 diff in time of closest approach (seconds)
daysec*(t_2029(A2_JPL+3σ_A2_JPL)-t_2029(A2_JPL-3σ_A2_JPL))

In [ ]:
# 3*σ_A2 diff in closest approach distance (km)
au*(rae_2029(A2_JPL+3σ_A2_JPL)-rae_2029(A2_JPL-3σ_A2_JPL))

In [ ]:
au*rae_2029(A2_JPL)

In [ ]:
#difference in time of April 2029 closest approach wrt JPL#199 (seconds)
(  ( jd0+t_2029(A2_JPL) ) - 2462240.407032288  )*daysec

In [ ]:
#diff. in geocentric range wrt #199 at closest approach (meters): April 2029
1e3au*(rea(sol.xvS1[47,:], params, sol.tvS1[47])(A2_JPL)-d_CA_2029_JPL199)

In [ ]:
# geocentric radial velocity at 2029 closest approach in au/day (should be close to zero)
r_dot_v = (xae_2029[1]*xae_2029[4]+xae_2029[2]*xae_2029[5]+xae_2029[3]*xae_2029[6])/rae_2029

In [ ]:
# geocentric radial velocity at 2029 closest approach in mm/s
1e6au*r_dot_v/daysec

# 2029 close approach: B-plane

In [ ]:
a_2029 = semimajoraxis(xae_2029...,μ[ea], 0.0)

In [ ]:
# asymptotic inbound velocity v_\infty (au/day)
v_infty = sqrt(μ[ea]/(-a_2029))

In [ ]:
# asymptotic inbound velocity v_\infty (km/sec)
# k-th order term has units (km/sec) * (10^-14 au/day^2)^{-k}
au*v_infty/daysec

In [ ]:
# h = r × v
hvec_2029 = cross(xae_2029[1:3], xae_2029[4:6])

In [ ]:
h_2029 = sqrt(hvec_2029[1]^2 + hvec_2029[2]^2 + hvec_2029[3]^2)

In [ ]:
# Laplace-Runge-Lenz (eccentricity) vector
# \vec{e} = (\vec{v} × \vec{h})/μ - \vec{r}/r
evec_2029 = cross(xae_2029[4:6], hvec_2029)/μ[ea] - xae_2029[1:3]/rae_2029

In [ ]:
# magnitude of eccentricity vector
sqrt(evec_2029[1]^2 + evec_2029[2]^2 + evec_2029[3]^2)

In [ ]:
# osculating eccentricity
e_2029 = eccentricity(xae_2029..., μ[ea], 0.0)

In [ ]:
# osculating eccentricity
i_2029 = inclination(xae_2029...)

In [ ]:
# Earth heliocentric semimajor axis
a_E = semimajoraxis(Apophis.kmsec2auday(earth_et((jd0-J2000+sol.tvS1[47]())*daysec)-sun_et((jd0-J2000+sol.tvS1[47]())*daysec))..., μ[su], 0.0)

In [ ]:
# periapsis position (unit vector)
P_v = evec_2029./e_2029;

In [ ]:
# periapsis velocity (unit vector)
Q_v = cross(hvec_2029, P_v)./h_2029;

In [ ]:
#inbound asymptote direction 
S_v = (P_v + (sqrt(e_2029^2 - 1))Q_v)/e_2029

In [ ]:
(1/e_2029)^2 + (sqrt(e_2029^2 - 1)/e_2029)^2

In [ ]:
# B-vector: "vector from the planet center to the intersection between the B-plane and the asymptote"
Bvec_2029 = cross(S_v, hvec_2029)./v_infty

In [ ]:
# impact parameter B
B_2029 = sqrt( Bvec_2029[1]^2 + Bvec_2029[2]^2 + Bvec_2029[3]^2 )

In [ ]:
V_v = [zero(B_2029), zero(B_2029), -one(B_2029)]

In [ ]:
T_v_unnormalized = cross(V_v, S_v)
T_v_norm = sqrt(T_v_unnormalized[1]^2 + T_v_unnormalized[2]^2 + T_v_unnormalized[3]^2)
T_v = T_v_unnormalized./T_v_norm

In [ ]:
R_v = cross(S_v, T_v)

In [ ]:
#B.R
B_dot_R = dot(T_v, hvec_2029)/v_infty

In [ ]:
#B.T
B_dot_T = -dot(R_v, hvec_2029)/v_infty

In [ ]:
B_2029

In [ ]:
# Earth impact cross section; "critical" B
# derived from conservation of energy and angular momentum
#  = what impact parameter B corresponds to a grazing impact, given planet radius and mass, and asymptotic vel v_infty?
# if actual B is equal or less to this, then impact happens
b_E_2029 = sqrt( 1 + (2μ[ea])/((PlanetaryEphemeris.RE/au)*(v_infty)^2) )

In [ ]:
fact_p = 3
fact_m = 142
A2v = (A2_JPL-σ_A2_JPL*fact_m):(A2_JPL+σ_A2_JPL*fact_p)

In [ ]:
plot(B_dot_R.(A2v)*au/PlanetaryEphemeris.RE, B_dot_T.(A2v)*au/PlanetaryEphemeris.RE, aspectratio=1, label="Apophis (JT)")
plot!(cos.(0:0.1:2pi+0.1), sin.(0:0.1:2pi+0.1), label="Earth radius")
plot!(b_E_2029()*cos.(0:0.1:2pi+0.1), b_E_2029()*sin.(0:0.1:2pi+0.1), label="Earth cross section", leg=:bottomright)
xlabel!("B.R [Earth radii]")
ylabel!("B.T [Earth radii]")
title!("B-plane; v=-e_z (Farnocchia et al 2019 convention)")
#xlims!(-4, 3)
#ylims!(-8, 3)

In [ ]:
#B_2029.(A2v)/(PlanetaryEphemeris.RE/au)

In [ ]:
plot(A2v, B_2029.(A2v)/(PlanetaryEphemeris.RE/au), label="B(A_2)")
plot!(A2v, map(x->1.0, A2v), label="Earth radius")
plot!(A2v, b_E_2029.(A2v), label="Earth cross section")
ylims!(0,12)
xlims!(-15,1)
ylabel!("B_2029 [Earth radii]")
xlabel!("A2×10^14 [au/day^2]")
title!("B as a function of A_2")

In [ ]:
# lunar distance in Earth radii
PlanetaryEphemeris.ld/PlanetaryEphemeris.RE

In [ ]:
# 2029 nominal CA distance in lunar distances
rae_2029(A2_lsqfit_deldop)*au/PlanetaryEphemeris.ld

# B-plane: Öpik's frame

Valsecchi (2005): Assume that the ratio of the periods of the small body and of the Earth is $k/h$. Then, following a first encounter, after $h$ heliocentric revolutions of the small body and $k$ revolutions of the Earth, both the Earth and the small body will be back to the same position of the previous encounter, performing a resonant return.

In [ ]:
# Sun's barycentric cartesian state at Apophis 2029 CA as a function of A2
xs_2029 = ss16asteph_2029[union(3su-2:3su,3(Apophis.N-1+su)-2:3(Apophis.N-1+su))];

In [ ]:
# Earth's heliocentric cartesian state at Apophis 2029 CA as a function of A2
xes_2029 = xe_2029 - xs_2029;

In [ ]:
# Earth's heliocentric velocity at Apophis 2029 CA as a function of A2 (au/day)
v_pl_2029 = xes_2029[4:6]

In [ ]:
#ξ-axis is essentially the MOID (Valsecchi et al, 2003)
ξ_v_unnormalized = cross(v_pl_2029, S_v)
ξ_v_norm = sqrt(ξ_v_unnormalized[1]^2 + ξ_v_unnormalized[2]^2 + ξ_v_unnormalized[3]^2)
ξ_v = ξ_v_unnormalized./ξ_v_norm

In [ ]:
#ζ-axis: delay/advance in CA time (Valsecchi et al, 2003)
ζ_v = -cross(S_v, ξ_v)

In [ ]:
B_dot_ξ = dot(Bvec_2029, ξ_v);

In [ ]:
B_dot_ζ = dot(Bvec_2029, ζ_v);

In [ ]:
B_dot_ξ*au/PlanetaryEphemeris.RE

In [ ]:
B_dot_ζ*au/PlanetaryEphemeris.RE

# Valsecchi circles

We compute Valsecchi circles as a function of the components of the unperturbed planetocentric velocity vector $\vec{U}$ of Apophis in a coordinate frame $(X,Y,Z)$ such that the $Y$-axis is along the heliocentric velocity vector of the planet. It is enough to have the $U_y=U\cos\theta$ component and the norm $U=|\vec{U}|$, where $\theta$ is the angle between the $Y$-axis and $\vec{U}$. Furthermore, $\vec{U}$ should be in units such that the planet's heliocentric velocity is 1.

In [ ]:
S_v_nom = S_v(A2_lsqfit_deldop) # small-body unperturbed planetocentric velocity unit vector

In [ ]:
norm(S_v_nom)

In [ ]:
ves_2029 = xes_2029(A2_lsqfit_deldop)[4:6]*au/daysec # Earth's velocity, km/s
ves_2029_norm = norm(ves_2029) # Earth's speed, km/s

In [ ]:
ves_2029_unit = ves_2029/ves_2029_norm # Earth's velocity unit vector

In [ ]:
# angle between Y-axis and \vec{U}
cosθ_ = dot(S_v_nom, ves_2029_unit)

In [ ]:
v_infty_kms = v_infty(A2_lsqfit_deldop)*au/daysec # Apophis unperturbed speed, km/sec

In [ ]:
# The norm of \vec{U} in appropriate units
U_norm_ = v_infty_kms/ves_2029_norm

In [ ]:
# U_y
U_y_ = U_norm_*cosθ_

In [ ]:
R__, D__ = valsecchi_circle(U_y_, U_norm_, 7, 6)
D_Valsecchi_7_6 = D__/(PlanetaryEphemeris.RE/au)
R_Valsecchi_7_6 = R__/(PlanetaryEphemeris.RE/au)
D_Valsecchi_7_6, R_Valsecchi_7_6

In [ ]:
R__, D__ = valsecchi_circle(U_y_, U_norm_, 6, 5)
D_Valsecchi_6_5 = D__/(PlanetaryEphemeris.RE/au)
R_Valsecchi_6_5 = R__/(PlanetaryEphemeris.RE/au)
D_Valsecchi_6_5, R_Valsecchi_6_5

In [ ]:
R__, D__ = valsecchi_circle(U_y_, U_norm_, 8, 7)
D_Valsecchi_8_7 = D__/(PlanetaryEphemeris.RE/au)
R_Valsecchi_8_7 = R__/(PlanetaryEphemeris.RE/au)
D_Valsecchi_8_7, R_Valsecchi_8_7

In [ ]:
A2_JPL

In [ ]:
σ_A2_JPL

In [ ]:
fact_p_ξζ = 5 #10 #30 #10
fact_m_ξζ = 5 #10 #200 #143 #3
A2v_ξζ = (A2_JPL-σ_A2_JPL*fact_m_ξζ):(A2_JPL+σ_A2_JPL*fact_p_ξζ)

In [ ]:
plot(B_dot_ξ.(A2v_ξζ)*au/PlanetaryEphemeris.RE, B_dot_ζ.(A2v_ξζ)*au/PlanetaryEphemeris.RE, aspectratio=1, label="Apophis (JT)")
scatter!([B_dot_ξ(A2_JPL)*au/PlanetaryEphemeris.RE], [B_dot_ζ(A2_JPL)*au/PlanetaryEphemeris.RE], label="nominal", marker=:xcross)
plot!(cos.(0:0.1:2pi+0.1), sin.(0:0.1:2pi+0.1), label="Earth radius")
plot!(b_E_2029()*cos.(0:0.1:2pi+0.1), b_E_2029()*sin.(0:0.1:2pi+0.1), label="Earth cross section")
plot!(
    R_Valsecchi_8_7*cos.(0:0.1:2pi+0.1), 
    D_Valsecchi_8_7.+R_Valsecchi_8_7*sin.(0:0.1:2pi+0.1), 
    label="8:7 res. Valsecchi circle",
    leg=:outertopright,
    aspectratio=0.6
)
plot!(
    R_Valsecchi_7_6*cos.(0:0.1:2pi+0.1), 
    D_Valsecchi_7_6.+R_Valsecchi_7_6*sin.(0:0.1:2pi+0.1), 
    label="7:6 res. Valsecchi circle",
)
plot!(
    R_Valsecchi_6_5*cos.(0:0.1:2pi+0.1), 
    D_Valsecchi_6_5.+R_Valsecchi_6_5*sin.(0:0.1:2pi+0.1), 
    label="6:5 res. Valsecchi circle",
)
xlabel!("B.xi [Earth radii]")
ylabel!("B.zeta [Earth radii]")
title!("B-plane (Öpik's frame)")
xlims!(-5, 5)
xticks!(-5:5)
ylims!(-0.1, 9)
yticks!(0:9)

In [ ]:
PlanetaryEphemeris.RE

In [ ]:
#gr()

- Positive $A_2$ value seems to be related to a prograde rotation state (Farnocchia et al 2013b).
- Why do Valsecchi circles are not identical to Farnocchia et al (2013b), even if B-plane projection of
    nominal solution is almost identical? In our plots the nominal solution is *inside* the 7:6 Valsecchi
    circle, whereas Farnocchia et al (2013b) nominal is *outside* the 7:6 Valsecchi circle --> That's because it's better to compute Valsecchi circles directly from unperturbed planetocentric velocity components, instead of using orbital elements (approximations in Öpik's theory, etc, are sources of error)
- Why is the inclination wrt ecliptic used instead of equatorial? --> Those are the elements used by Valsecchi

## Resonances

In [ ]:
# pre-2029 encounter Apophis heliocentric semimajor axis
a_Ap_pre_2029 = semimajoraxis(sol.xvS1[1,1:6]-Apophis.kmsec2auday(sun_et((jd0-J2000+sol.tvS1[1])*daysec))..., μ[su], 0.0)

In [ ]:
# pre-2029 encounter Apophis heliocentric semimajor axis
i_Ap_pre_2029 = inclination(sol.xvS1[1,1:6]-Apophis.kmsec2auday(sun_et((jd0-J2000+sol.tvS1[1])*daysec))...)

In [ ]:
# post-2029 encounter semimajor axis
a_Ap_post_2029 = semimajoraxis(sol.xvS1[end,1:6]-Apophis.kmsec2auday(sun_et((jd0-J2000+sol.tvS1[end])*daysec))..., μ[su], 0.0)

In [ ]:
a_Ap_pre_2029/a_E

In [ ]:
a_Ap_post_2029(A2_lsqfit_deldop)/a_E

In [ ]:
for i in 1:8
    k = i; h = i+1
    #@show k, h, (k/h)^(2/3), (h/k)^(2/3)
    println("k=", k, ", h=", h, ", (k/h)^(2/3)=", (k/h)^(2/3))
end

In [ ]:
for i in 2:9
    k = i; h = i-1
    #@show k, h, (k/h)^(2/3), (h/k)^(2/3)
    println("k=", k, ", h=", h, ", (k/h)^(2/3)=", (k/h)^(2/3))
end

In [ ]:
#nominal 2029 CA distance
0.000252172816142565

## Observations predictions 2021/2029

In [ ]:
apophis_data[end].rcvr

In [ ]:
typeof(ans)

In [ ]:
?delay_doppler

In [ ]:
tdelay_2021, dshift_2021 = delay_doppler(
    apophis_data[end].rcvr,
    DateTime(2021,1,1),
    apophis_data[end].freq,
    niters,
    false
)

In [ ]:
import EarthOrientation

In [ ]:
EarthOrientation.update()

## Plots for paper

In [ ]:
#gr()

In [ ]:
pgfplots(grid=:false, framestyle=:box)
font_size = 14

In [ ]:
A2_plot = 0.0
scatter(
    tv_jpl_utc_jul[delay_index].-t0, tdelay_v1[delay_index](A2_plot),
    label="predicted (\$A_2 = $A2_plot \\times 10^{-14} \$ au/d\$^2\$)",
    marker=:cross,
    markersize=7,
)
#scatter!(
#    tv_jpl_utc_jul[delay_index].-t0,
#    tdelay_JPL_v1[delay_index],
#    label="predicted (JPL DE430+s199)",
#    marker=:xcross
#)
scatter!(
    tv_jpl_utc_jul[delay_index].-t0,
    tdelay_jpl_obs,
    label="observed (Brozovic et al., 2018)",
    legend=:topleft,
    marker=:xcross,
    markersize=7,
    yerror=dshift_jpl_obs_sigma,
    tickfont=font(font_size),
    guidefont=font(font_size),
    legendfont = font(font_size-4),
    xticks = (1540:30:1640, ()),
    annotations=( 1640, 2.6e8, "(a)")
)
#title!("Time delay vs time")
ylabel!("Time delay [\$\\mu\$s]")
#xlims!(1540, 1645)
#xticks!(1540:20:1640, string.(Date.(julian2datetime.(t0 .+ collect(1540:20:1640)))))
xlims!(1540, 1645)
ylims!(7.5e7,2.75e8)
#xticks!(1540:30:1640, string.(Date.(julian2datetime.(t0 .+ collect(1540:30:1640)))))
#xlabel!("Time")

In [ ]:
A2_plot = 0.0
scatter(
    tv_jpl_utc_jul[doppler_index].-t0, dshift_v1[doppler_index](A2_plot),
    label="predicted (\$A_2 = $A2_plot \\times 10^{-14} \$ au/d\$^2\$)",
    marker=:cross,
    markersize=7,
    tickfont=font(font_size),
    guidefont=font(font_size),
    legendfont = font(font_size-4)
)
#scatter!(
#    tv_jpl_utc_jul[doppler_index].-t0,
#    dshift_JPL_v1[doppler_index],
#    label="predicted (JPL DE430+s199)",
#    marker=:xcross
#)
scatter!(
    tv_jpl_utc_jul[doppler_index].-t0,
    dshift_jpl_obs,
    label="observed (Brozovic et al., 2018)",
    legend=:bottomleft,
    marker=:xcross,
    markersize=7,
    yerror=dshift_jpl_obs_sigma,
    annotations=( 1640, 50000, "(b)")
)
title!("Doppler shift vs time")
ylabel!("Doppler shift [Hz]")
xlims!(1540, 1645)
xticks!(1540:30:1640, string.(Date.(julian2datetime.(t0 .+ collect(1540:30:1640)))))
xlabel!("Time")

## Plots for DDA poster

In [ ]:
pgfplots(grid=:false, framestyle=:box)

In [ ]:
font_size = 14

In [ ]:
A2_plot = -5.74846133680956 # A2_lsqfit_deldop
A2_label_str = @sprintf "\$A_2^* = -5.74 \\times 10^{-14} \$ au/d\$^2\$"
plot(
    tv_jpl_utc_jul[delay_index].-t0,
    residual_td(), yerror = tdelay_jpl_obs_sigma,
    marker=:cross,
    label = "\$A_2 = 0\$ au/d\$^2\$",
    legend = :topright,
    tickfont=font(font_size),
    guidefont=font(font_size),
    legendfont = font(font_size-4),
    xticks = (1540:30:1640, ()),
    annotations=( 1555, 300, "(a)")
)
for i in 1:4
    a2_lbl_str_i = "\$A_2 = -$i \\times 10^{-14}\$ au/d\$^2\$"
    plot!(
    tv_jpl_utc_jul[delay_index].-t0,
    residual_td(-i), yerror = tdelay_jpl_obs_sigma,
    marker=:cross,
    label = a2_lbl_str_i,
    legend = :topright
)
end
plot!(
    tv_jpl_utc_jul[delay_index].-t0,
    residual_td(A2_plot), yerror = tdelay_jpl_obs_sigma,
    marker=:cross,
    label = A2_label_str,
    legend = :topright
)
plot!(
    tv_jpl_utc_jul[delay_index].-t0,
    residual_td(-7), yerror = tdelay_jpl_obs_sigma,
    marker=:cross,
    label = "\$A_2 = -7 \\times 10^{-14} \$ au/d\$^2\$",
    legend = :topright
)
ylabel!("Time delay O-C residual \$ [\\mu\\mathrm{s}] \$")
xlims!(1540, 1645)
#ylims!(-5,4)
ylims!(-250,350)
#xticks!(1540:30:1640, string.(Date.(julian2datetime.(t0 .+ collect(1540:30:1640)))))
#xlabel!("Calendar date")
#savefig("delay_residuals_A2.pdf")

In [ ]:
A2_plot = -5.74846133680956 # A2_lsqfit_deldop
A2_label_str = @sprintf "\$A_2^* = -5.74846 \\times 10^{-14} \$ au/d\$^2\$"
plot(
    tv_jpl_utc_jul[doppler_index].-t0,
    residual_ds(), yerror = dshift_jpl_obs_sigma,
    marker=:cross,
    label = "\$A_2 = 0\$ au/d\$^2\$",
    legend = :bottomright,
    tickfont=font(font_size),
    guidefont=font(font_size),
    legendfont = font(font_size-4),
    annotations=( 1555, 0.95, "(b)")
)
for i in 3:3
    a2_lbl_str_i = "\$A_2 = -$i \\times 10^{-14} \$ au/d\$^2\$"
    plot!(
    tv_jpl_utc_jul[doppler_index].-t0,
    residual_ds(-i), yerror = dshift_jpl_obs_sigma,
    marker=:cross,
    label = a2_lbl_str_i,
    #legend = :topright,
    color=3+1
)
end
plot!(
    tv_jpl_utc_jul[doppler_index].-t0,
    residual_ds(A2_plot), yerror = dshift_jpl_obs_sigma,
    marker=:cross,
    label = A2_label_str,
    #legend = :topright,
    color=6
)
plot!(
    tv_jpl_utc_jul[doppler_index].-t0,
    residual_ds(-8), yerror = dshift_jpl_obs_sigma,
    marker=:cross,
    label = "\$A_2 = -7 \\times 10^{-14} \$ au/d\$^2\$",
    #legend = :topright,
    color=7
)
ylabel!("Doppler shift O-C residual \$ [\\mathrm{Hz}] \$")
xlims!(1540, 1645)
#ylims!(-5,4)
#ylims!(-250,350)
xticks!(1540:30:1640, string.(Date.(julian2datetime.(t0 .+ collect(1540:30:1640)))))
xlabel!("Calendar date")
#savefig("doppler_residuals_A2.pdf")

In [ ]:
plot(
    A2_vals,
    Q_A2_deldop.(A2_vals),
    leg=false,
    tickfont=font(font_size),
    guidefont=font(font_size),
    legendfont = font(font_size-4)
)
xlabel!("\$A_2 \\times 10^{14} \$ [au/d\$^2]\$")
ylabel!("\$Q(A_2)\$")
#title!("Mean square residual vs A2")
ylims!(-1, 11)
xlims!(-5.9, -5.5)
#savefig("Q_vs_A2.pdf")